# Procesamiento en PySpark

Configuración del context para usar PySpark

In [4]:
import sys
print(sys.executable)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/bin/python3

In [11]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1725594577268_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1725594577268_0002,pyspark,idle,Link,Link,None,✔


In [16]:
sc.list_packages()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ------------
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
click                      8.1.7
docutils                   0.14
jmespath                   1.0.1
joblib                     1.3.2
lockfile                   0.11.0
lxml                       4.9.3
mysqlclient                1.4.2
nltk                       3.8.1
nose                       1.3.4
numpy                      1.20.0
pip                        20.2.2
py-dateutil                2.2
pystache                   0.5.4
python-daemon              2.2.3
python37-sagemaker-pyspark 1.4.2
pytz                       2023.3.post1
PyYAML                     5.4.1
regex                      2021.11.10
setuptools                 28.8.0
simplejson                 3.2.0
six                        1.13.0
tqdm                       4.66.1
wheel                      0.29.0
windmill                   1.6


Conexión con la base de datos

In [23]:
df = spark.sql("show databases")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
| namespace|
+----------+
|   default|
|    labsdb|
|usernamedb|
+----------+

Consultas de las tablas

# Tabla Atm_con: Hacemos un select para traernos todos los campos y conocer la información con la que contamos

In [3]:
df = spark.sql("select * from labsdb.atm_con")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+-----+
|objectid|country|iso2|iso3|           indicator|                unit|          source|            cts_code|            cts_name| cts_full_descriptor|                date|value|
+--------+-------+----+----+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+-----+
|    null|Country|ISO2|ISO3|           Indicator|                Unit|          Source|            CTS_Code|            CTS_Name| CTS_Full_Descriptor|                Date| null|
|       1|  World|    | WLD|Monthly Atmospher...|   Parts Per Million|"Dr. Pieter Tans| National Oceanic...| Trends in Atmosp...| Scripps Institut...| Carbon Dioxide M...| null|
|       2|  World|    | WLD|Monthly Atmospher...|   Parts Per Million|"Dr. Pieter Tans| National Oceanic...| T

Conteo de palabras para conocer cuales son las palabras mas repetidas en la tabla de atmosfera (atm_con):

In [1]:
files_rdd = sc.textFile("s3://trustedtrabajo1/trusted/atm_con.txt")
wc_unsort = files_rdd.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
wc = wc_unsort.sortBy(lambda a: -a[1])
for tupla in wc.take(10):
        print(tupla)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1725738867665_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Carbon', 7910)
('Atmospheric', 7910)
('Dioxide', 7910)
('and', 4746)
('Concentrations', 3961)
('Climate', 3164)
('Dr.', 3164)
('on', 2367)
('World', 1582)
('National', 1582)

In [36]:
from pyspark.sql import SparkSession

# spark local:
spark = SparkSession.builder.master("local[*]").getOrCreate()

# spark conectado con Amazon AWS:

spark = SparkSession.builder.appName("data_processing").master("local[*]").config("spark.driver.memory","16G").config("spark.driver.maxResultSize", "0").config("spark.kryoserializer.buffer.max", "2000M").config("spark.jars.packages", "com.amazonaws:aws-java-sdk:1.11.1034,org.apache.hadoop:hadoop-aws:3.3.4").config('fs.s3a.access.key', "AWS_ACCESS_KEY").config('fs.s3a.secret.key', "AWS_SECRET_KEY").config('fs.s3a.session.token',"AWS_SESSION_TOKEN").config('fs.s3a.path.style.access', 'true').config('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem').config('fs.s3a.endpoint', 's3.amazonaws.com').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Aca estraemos las datablas directamente desde el bucket de S3:

In [37]:
df=spark.read.csv('s3://rawtrabajo1/raw/ATM_CON/atm_con.csv',inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(df.columns) #Mostramos los nombres de las columnas que contiene la tabla
print(len(df.columns)) #Numero de columnas
print(df.count()) #Cantidad de datos que contiene el dataframe
print((df.count(),len(df.columns))) #Tamaño del dataframe (cantidad de columnas y filas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['objectid', 'country', 'iso2', 'iso3', 'indicator', 'unit', 'source', 'cts_code', 'cts_name', 'cts_full_descriptor', 'date', 'value']
12
1583
(1583, 12)

Mostramos el esquema de como esta creada la tabla: las columnas que contiene, el tipo de dato de cada columna y si cuenta con nulos o no:

In [7]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- objectid: long (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- indicator: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- source: string (nullable = true)
 |-- cts_code: string (nullable = true)
 |-- cts_name: string (nullable = true)
 |-- cts_full_descriptor: string (nullable = true)
 |-- date: string (nullable = true)
 |-- value: string (nullable = true)

df.printSchema() 

In [43]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+-------+------+
|ObjectId|Country|ISO2|ISO3|           Indicator|             Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|   Date| Value|
+--------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+-------+------+
|       1|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1958M03|315.71|
|       2|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1958M04|317.45|
|       3|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1958M05|317.51|
|       4|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr.

Hacemos un select para conocer los paises, el indicador y el valor de las concentraciones de carbono en la atmosfera:

In [8]:
df.select('Country','Indicator', 'value').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------+
|Country|           Indicator| value|
+-------+--------------------+------+
|Country|           Indicator| Value|
|  World|Monthly Atmospher...|315.71|
|  World|Monthly Atmospher...|317.45|
|  World|Monthly Atmospher...|317.51|
|  World|Monthly Atmospher...|317.27|
+-------+--------------------+------+
only showing top 5 rows

Sacamos estadisticos para analizar el comportamiento de cada variable como la cantidad de datos, el promedio, valores minimos y maximos, para las variables string los muestra como null:

In [10]:
df.describe().show() #estadisticos para cada variable

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+-------+------------------+
|summary|         objectid|country|iso2|iso3|           indicator|             unit|              source|cts_code|            cts_name| cts_full_descriptor|   date|             value|
+-------+-----------------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+-------+------------------+
|  count|             1582|   1583|1583|1583|                1583|             1583|                1583|    1583|                1583|                1583|   1583|              1583|
|   mean|            791.5|   null|null|null|                null|             null|                null|    null|                null|                null|   null|181.23111251580278|
| stddev|456.8283776941475|   null|null|null|                null|             n

Hacemos procesamiento de las variables:

In [12]:
from pyspark.sql.types import StringType,DoubleType,IntegerType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df.withColumn('value_double',df['value'].cast(DoubleType())).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+-------------------------------------------------+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-----------------------------------------+-------------------------------------------------------------------------------------------+-------+------+------------+
|objectid|country|iso2|iso3|indicator                                        |unit             |source                                                                                                                                                                                                                                                         |cts_code|cts_name                                 |cts_full_descriptor                                                      

In [15]:
df.filter(df['unit']=='Percent').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|objectid|country|iso2|iso3|           indicator|   unit|              source|cts_code|            cts_name| cts_full_descriptor|   date|value|
+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|      14|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1959M03|  0.3|
|      16|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1959M04| 0.09|
|      18|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1959M05| 0.25|
|      20|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1959M06

In [18]:
df.filter(df['unit']=='Percent').select('Country','indicator','unit', 'date','value').show() #porcentaje de 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------+-------+-----+
|Country|           indicator|   unit|   date|value|
+-------+--------------------+-------+-------+-----+
|  World|Monthly Atmospher...|Percent|1959M03|  0.3|
|  World|Monthly Atmospher...|Percent|1959M04| 0.09|
|  World|Monthly Atmospher...|Percent|1959M05| 0.25|
|  World|Monthly Atmospher...|Percent|1959M06| 0.28|
|  World|Monthly Atmospher...|Percent|1959M07| 0.21|
|  World|Monthly Atmospher...|Percent|1959M08|-0.04|
|  World|Monthly Atmospher...|Percent|1959M09|  0.2|
|  World|Monthly Atmospher...|Percent|1959M10| 0.29|
|  World|Monthly Atmospher...|Percent|1959M11| 0.47|
|  World|Monthly Atmospher...|Percent|1959M12| 0.29|
|  World|Monthly Atmospher...|Percent|1960M01| 0.27|
|  World|Monthly Atmospher...|Percent|1960M02| 0.15|
|  World|Monthly Atmospher...|Percent|1960M03| 0.29|
|  World|Monthly Atmospher...|Percent|1960M04| 0.41|
|  World|Monthly Atmospher...|Percent|1960M05| 0.55|
|  World|Monthly Atmospher...|Percent|1960M06|

In [19]:
df.filter(df['unit']=='Percent').filter(df['value'] > 0.5 ).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|objectid|country|iso2|iso3|           indicator|   unit|              source|cts_code|            cts_name| cts_full_descriptor|   date|value|
+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|      42|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1960M05| 0.55|
|      46|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1960M07| 0.52|
|     186|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1966M05| 0.59|
|     188|  World|    | WLD|Monthly Atmospher...|Percent|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1966M06

In [25]:
df.select('indicator').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           indicator|
+--------------------+
|           Indicator|
|Monthly Atmospher...|
|Monthly Atmospher...|
+--------------------+

In [26]:
df.select('indicator').distinct().count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3

In [24]:
df.groupBy('indicator').count().show(5,False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------+-----+
|indicator                                                                        |count|
+---------------------------------------------------------------------------------+-----+
|Indicator                                                                        |1    |
|Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change|785  |
|Monthly Atmospheric Carbon Dioxide Concentrations                                |797  |
+---------------------------------------------------------------------------------+-----+

In [27]:
df.groupBy('indicator').count().orderBy('count',ascending=False).show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------+-----+
|indicator                                                                        |count|
+---------------------------------------------------------------------------------+-----+
|Monthly Atmospheric Carbon Dioxide Concentrations                                |797  |
|Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change|785  |
|Indicator                                                                        |1    |
+---------------------------------------------------------------------------------+-----+

In [34]:
df.groupBy('date').mean().show(5,False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
|date   |avg(objectid)|
+-------+-------------+
|1960M06|43.5         |
|1963M06|115.5        |
|1962M06|91.5         |
|1962M11|101.5        |
|1963M05|113.5        |
+-------+-------------+
only showing top 5 rows

In [35]:
df.groupBy('date').sum().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
|date   |sum(objectid)|
+-------+-------------+
|1960M06|87           |
|1963M06|231          |
|1962M06|183          |
|1962M11|203          |
|1963M05|227          |
+-------+-------------+
only showing top 5 rows

In [36]:
df.groupBy('date').max().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
|date   |max(objectid)|
+-------+-------------+
|1960M06|44           |
|1963M06|116          |
|1962M06|92           |
|1962M11|102          |
|1963M05|114          |
+-------+-------------+
only showing top 5 rows

In [37]:
df.groupBy('date').min().show(5,False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
|date   |min(objectid)|
+-------+-------------+
|1960M06|43           |
|1963M06|115          |
|1962M06|91           |
|1962M11|101          |
|1963M05|113          |
+-------+-------------+
only showing top 5 rows

In [38]:
df.groupBy('date').agg({'value':'sum'}).show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+
|date   |sum(value)|
+-------+----------+
|1960M06|320.03    |
|1963M06|321.76    |
|1962M06|320.89    |
|1962M11|316.88    |
|1963M05|322.62    |
+-------+----------+
only showing top 5 rows

In [50]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
def cant_carbono(value):
    try:
        value = float(value) 
        if value < 412:
            return 'Bajo carbono'
        else:
            return 'Alto carbono'
    except ValueError:
        return 'Invalid value'  

cant_udf = udf(cant_carbono, StringType())
df = df.withColumn('cant_carbono', cant_udf(df['value']))
df.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+-------------------------------------------------+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-----------------------------------------+-------------------------------------------------------------------------------------------+-------+------+-------------+
|objectid|country|iso2|iso3|indicator                                        |unit             |source                                                                                                                                                                                                                                                         |cts_code|cts_name                                 |cts_full_descriptor                                                     

In [53]:
from pyspark.sql.functions import pandas_udf, PandasUDFType


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
def remaining_yrs(date):
    yrs_left=100-date

    return yrs_left

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
#create udf using python function
length_udf = pandas_udf(remaining_yrs, IntegerType())
#apply pandas udf on dataframe
df.withColumn("yrs_left", length_udf(df['date'])).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Pandas >= 1.0.5 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1725738867665_0001/container_1725738867665_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py", line 336, in pandas_udf
    require_minimum_pandas_version()
  File "/mnt/yarn/usercache/livy/appcache/application_1725738867665_0001/container_1725738867665_0001_01_000001/pyspark.zip/pyspark/sql/pandas/utils.py", line 36, in require_minimum_pandas_version
    ) from raised_error
ImportError: Pandas >= 1.0.5 must be installed; however, it was not found.



In [69]:
# eliminar duplicados
print(df.count())
df=df.dropDuplicates()
print(df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1583
1583

In [71]:
#drop column of dataframe
df_new=df.drop('date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
df_new.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+------+------------+
|objectid|country|iso2|iso3|           indicator|             unit|              source|cts_code|            cts_name| cts_full_descriptor| value|cant_carbono|
+--------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+------+------------+
|       9|  World|    | WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|313.33|Bajo carbono|
|     185|  World|    | WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|324.08|Bajo carbono|
|      13|  World|    | WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|316.65|Bajo carbono|
|      28|  World|    | WLD|Monthly Atmo

In [77]:
 write_uri='s3://trustedtrabajo1/trusted/df_csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
write_uri='s3://bucket_name/df_parquet'
df.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o474.save.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: The specified bucket does not exist (Service: Amazon S3; Status Code: 404; Error Code: NoSuchBucket; Request ID: YR1XJ107ZTVTH663; S3 Extended Request ID: 5zh34DFXC5gdeWU7OsJ0yV/MBC3vKRW2O7L63K1Uiwn6CwyLwTqu0N6Qj2hw8JFn8PaySeD6ijU=; Proxy: null), S3 Extended Request ID: 5zh34DFXC5gdeWU7OsJ0yV/MBC3vKRW2O7L63K1Uiwn6CwyLwTqu0N6Qj2hw8JFn8PaySeD6ijU=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:423)
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.isFolderUsingFolderObject(Jets3tNativeFileSystemStore.java:249)
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.isFolder(Jets3tNativeFileSystemStore.java:212)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:554)
	at org.apache.hadoop.fs.FileS

# Tabla Temp_change: Hacemos un select para traernos todos los campos y conocer la información con la que contamos

In [2]:
df = spark.sql("select * from labsdb.temp_change")
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----+----+--------------------+--------------+--------------------+--------+--------------------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+-------+-------+--------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-------+-------+-------+-------+-------+--------+-------+-------+
|objectid|             country|iso2|iso3|           indicator|          unit|              source|cts_code|            cts_name| cts_full_descriptor|   f1961|   f1962|   f1963|   f1964|   f1965|   f1966|   f1967|   f1968|   f1969|   f1970|   f1971|   f1972|   f1973|   f1974|   f1975|   f1976|   f197

In [3]:
files_rdd = sc.textFile("s3://trustedtrabajo1/trusted/temp_change.txt")
wc_unsort = files_rdd.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
wc = wc_unsort.sortBy(lambda a: -a[1])
for tupla in wc.take(10):
        print(tupla)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Climate', 944)
('Temperature', 944)
('and', 482)
('the', 477)
('Surface', 472)
('Change', 472)
('to', 472)
('Change,', 472)
('of', 282)
('United', 241)

In [7]:
df=spark.read.csv('s3://rawtrabajo1/raw/TEMP_CHANGE/temp_change.csv',inferSchema=True,header=True, sep = ';')
print(df.columns) #Mostramos los nombres de las columnas que contiene la tabla
print(len(df.columns)) #Numero de columnas
print(df.count()) #Cantidad de datos que contiene el dataframe
print((df.count(),len(df.columns))) #Tamaño del dataframe (cantidad de columnas y filas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor', 'F1961', 'F1962', 'F1963', 'F1964', 'F1965', 'F1966', 'F1967', 'F1968', 'F1969', 'F1970', 'F1971', 'F1972', 'F1973', 'F1974', 'F1975', 'F1976', 'F1977', 'F1978', 'F1979', 'F1980', 'F1981', 'F1982', 'F1983', 'F1984', 'F1985', 'F1986', 'F1987', 'F1988', 'F1989', 'F1990', 'F1991', 'F1992', 'F1993', 'F1994', 'F1995', 'F1996', 'F1997', 'F1998', 'F1999', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023']
73
236
(236, 73)

In [8]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- F1961: double (nullable = true)
 |-- F1962: double (nullable = true)
 |-- F1963: double (nullable = true)
 |-- F1964: double (nullable = true)
 |-- F1965: double (nullable = true)
 |-- F1966: double (nullable = true)
 |-- F1967: double (nullable = true)
 |-- F1968: double (nullable = true)
 |-- F1969: double (nullable = true)
 |-- F1970: double (nullable = true)
 |-- F1971: double (nullable = true)
 |-- F1972: double (nullable = true)
 |-- F1973: double (nullable = true)
 |-- F1974: double (nullable = true)
 |-- F1975: double (nullable = true)
 |-- F1976: double (nullable = true)
 |-- 

In [25]:
df.head(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ObjectId=1, Country='Afghanistan, Islamic Rep. of', ISO2='AF', ISO3='AFG', Indicator='Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980', Unit='Degree Celsius', Source='Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from:\xa0https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.', CTS_Code='ECCS', CTS_Name='Surface Temperature Change', CTS_Full_Descriptor='Environment, Climate Change, Climate and Weather, Surface Temperature Change', F1961=-0.126, F1962=-0.173, F1963=0.844, F1964=-0.751, F1965=-0.22, F1966=0.239, F1967=-0.348, F1968=-0.398, F1969=-0.513, F1970=0.843, F1971=0.642, F1972=-1.095, F1973=0.264, F1974=-0.466, F1975=-0.442, F1976=-0.291, F1977=0.544, F1978=0.12, F1979=0.38, F1980=0.655, F1981=0.558, F1982=-0.286, F1983=0.24, F1984=0.252, F1985=0.397, F1986=-0.015, F1987=0.493, F1988=1.035, F

In [12]:
df.select('Country','Indicator', 'ISO2', 'ISO3', 'F2019','F2020', 'F2021', 'F2022', 'F2023').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----+------+-----+-----+-----+-----+-----+
|             Country|           Indicator|ISO2|  ISO3|F2019|F2020|F2021|F2022|F2023|
+--------------------+--------------------+----+------+-----+-----+-----+-----+-----+
|Afghanistan, Isla...|Temperature chang...|  AF|   AFG|0.991|0.587|1.475|2.154|1.956|
|              Africa|Temperature chang...|null|AFRTMP|1.297|  1.2|1.396|0.996|1.485|
|             Albania|Temperature chang...|  AL|   ALB|1.751|1.567|1.589|1.585|2.122|
|             Algeria|Temperature chang...|  DZ|   DZA|1.107|1.905| 2.36|1.776|2.274|
|      American Samoa|Temperature chang...|  AS|   ASM|1.209|1.101|0.938|0.925|0.951|
+--------------------+--------------------+----+------+-----+-----+-----+-----+-----+
only showing top 5 rows

In [21]:
df.describe(['F2019','F2020', 'F2021', 'F2022', 'F2023']).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|             F2019|             F2020|             F2021|             F2022|             F2023|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|               222|               220|               223|               223|               221|
|   mean| 1.429130630630631| 1.523422727272728|1.3098340807174886|  1.34895067264574|1.6473303167420823|
| stddev|0.4705686615270562|0.6246334438749978|0.5195694113857042|0.6784408943345038|0.6001559755099951|
|    min|             0.121|             0.059|            -0.569|            -0.698|             0.215|
|    max|             2.871|             3.669|             3.012|             3.512|             3.578|
+-------+------------------+------------------+------------------+------------------+------------------+

In [22]:
#filter country 
df['Country','Indicator', 'ISO2', 'ISO3', 'F2019','F2020', 'F2021', 'F2022', 'F2023'].filter(df['Country']=='Colombia').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----+----+-----+-----+-----+-----+-----+
| Country|           Indicator|ISO2|ISO3|F2019|F2020|F2021|F2022|F2023|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+
|Colombia|Temperature chang...|  CO| COL|1.278|1.338|0.824|0.671|1.265|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+

In [17]:
#filter the records 
df.filter(df['Country']=='Colombia').select('Country','Indicator', 'ISO2', 'ISO3', 'F2019','F2020', 'F2021', 'F2022', 'F2023').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----+----+-----+-----+-----+-----+-----+
| Country|           Indicator|ISO2|ISO3|F2019|F2020|F2021|F2022|F2023|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+
|Colombia|Temperature chang...|  CO| COL|1.278|1.338|0.824|0.671|1.265|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+

In [26]:
df['Country','Indicator', 'F2019','F2020', 'F2021', 'F2022', 'F2023'].filter(df['F2023'] >1.64).show()
#registros con diferencia en cambios de temperatura para el año 2023 mayor al promedio

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+-----+-----+-----+-----+
|             Country|           Indicator|F2019|F2020|F2021|F2022|F2023|
+--------------------+--------------------+-----+-----+-----+-----+-----+
|Afghanistan, Isla...|Temperature chang...|0.991|0.587|1.475|2.154|1.956|
|             Albania|Temperature chang...|1.751|1.567|1.589|1.585|2.122|
|             Algeria|Temperature chang...|1.107|1.905| 2.36|1.776|2.274|
|            Americas|Temperature chang...|1.229|1.303|1.558|1.075|1.772|
|Andorra, Principa...|Temperature chang...|1.884| 2.48| 1.45|3.155|2.786|
|    Armenia, Rep. of|Temperature chang...|1.864|1.954|2.183|1.845|2.258|
|                Asia|Temperature chang...|1.388|1.506| 1.54|1.723|1.778|
|             Austria|Temperature chang...|2.375|2.322| 1.41|2.538|2.572|
| Azerbaijan, Rep. of|Temperature chang...|1.878|1.889|1.958|2.029| 2.16|
|        Bahamas, The|Temperature chang...|  1.6|1.715|0.714|1.304|1.688|
| Bahrain, Kingdom of|Temperature chan

In [27]:
#los diferentes paises
df.select('Country').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|             Country|
+--------------------+
|Iran, Islamic Rep...|
| Uzbekistan, Rep. of|
|              Guyana|
|          Czech Rep.|
| San Marino, Rep. of|
|        Saint Helena|
|Central African Rep.|
|           Argentina|
|              Angola|
|    Netherlands, The|
|             Albania|
|           Nicaragua|
|                Peru|
|             Somalia|
|          Martinique|
|            Suriname|
|              Norway|
|Ethiopia, The Fed...|
|          Guadeloupe|
|Micronesia, Feder...|
+--------------------+
only showing top 20 rows

In [28]:
#cantidad de paises distintos
df.select('Country').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

236

Los datos con los que contamos en la tabla de cambio de temperatura ya vienen agrupados, por lo que tenemos un solo indicador y todos los datos por lo que no tiene sentido usar la funcion df.groupBy()


In [31]:
#normal function 
def price_range(brand):
    if brand in ['Samsung','Apple']:
        return 'High Price'
    elif brand =='MI':
        return 'Mid Price'
    else:
        return 'Low Price'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'mean'
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1725759676569_0001/container_1725759676569_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 2979, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name)
AttributeError: 'DataFrame' object has no attribute 'mean'

